### This notebook is for analyzing the steps during processing data. It contains a lot more documentation and code than the original script. 

In [ ]:
import numpy as np
import pandas as pd
import os
import time
from tqdm import tqdm

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import string

# Processing Optimization 
from functools import cache


In [ ]:
if not os.path.exists("Datasets/stopwords_ua_set.txt"):
      !wget -P"Datasets/" https://raw.githubusercontent.com/skupriienko/Ukrainian-Stopwords/refs/heads/master/stopwords_ua_set.txt

with open('Datasets/stopwords_ua_set.txt', 'r') as file:
    ukrainian_stop_words = file.read().splitlines()[0]

keys_to_filter = os.getenv('KEYS_TO_FILTER').split(',')
concatenated_path = os.getenv('CONCATENATED_PATH')
dataset_path = "Datasets/concatenated.csv"
english_topwords = set(stopwords.words('english'))

# word2vec_path = "Models/glove-global-vectors-for-word-representation.zip" 
# glove_100d_path = "Models/glove.6B.100d.txt"
# if not os.path.exists(word2vec_path) or not os.path.exists(glove_100d_path):
#     print("Downloading word2vec for data augmentation...")
#     !kaggle datasets download -d rtatman/glove-global-vectors-for-word-representation -p Models/
#     !unzip Models/glove-global-vectors-for-word-representation.zip -d Models
#     os.remove("Models/glove-global-vectors-for-word-representation.zip")
#     os.remove("Models/glove.6B.200d.txt")

In [ ]:
dataset = pd.read_csv(dataset_path)
dataset = pd.DataFrame(dataset)
dataset.head(100)

In [ ]:
def remove_urls(text):
      return re.sub(r'http\S+', 'redacted', text)
# For non-english datasets
def remove_english_words(text):
    # Looks for all English words and removes them.
    pattern = r'\b[a-zA-Z]+\b'
    return re.sub(pattern, '', text)
def delete_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text
def remove_mention(text):
  mention_regex = r"@\w+"
  return re.sub(mention_regex, "/mention", text)
def redact_email(text): 
    return re.sub(r'\S+@\S+', '/email', text)
# def remove_password(text): 
#     copy_text = text
#     pass_pattern = r'[A-Za-z0-9@#$%^&+=]{8,}'
#     text_ = re.sub(pass_pattern, '', text)
#     return text_
def remove_whitespace(text):
    return  " ".join(text.split())
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
def sen_len_threshold(text, char_min=16, char_limit=512): # Can be used for better tuning. 
    text = str(text)
    # Removes sentences if between char_min and char_limit.
    clean_text = text if char_min <= len(text) <= char_limit else None
    return clean_text

In [ ]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)

In [ ]:
def filter_sensitive_words(sentence, replacement='CENSORED', keys_to_filter=keys_to_filter):
    """
    Create a list of sensitive words 'keys_to_filter' from .env file 
    Replaces sensitive for you words with 'CENSORED'

    Parameters: 
        sentence 
        replacement: str = words that will be substituted instead of the sensitive words   
    """
    words = set(keys_to_filter)
    sentence_words = word_tokenize(sentence)
    
    modified_sentence = [
        replacement if word in words else word for word in sentence_words
    ]
    
    # Join the list back into a sentence
    return ' '.join(modified_sentence)
    
    return sentence

In [ ]:
# Since " " rows don't count as NAN, we should identify them by ourselves.
def drop_space_rows(df: pd.DataFrame, column: str ="Message") -> pd.DataFrame:
      """Identifies and drops ' ' rows in the DataFrame"""
      space_rows = (df[column] == ' ')| (df[column] == '')
      df_filtered = df[~pd.Series(space_rows)].reset_index(drop=True)

      return df_filtered

In [ ]:
def preprocess_data(text):
      text = remove_english_words(text)
      text = redact_email(text)
      text = remove_urls(text)
      text = remove_mention(text)
      text = delete_html_tags(text)
      text = filter_sensitive_words(text)
      text = remove_whitespace(text)
      
      return text

In [ ]:
def preprocess_dataset(df: pd.DataFrame) -> pd.DataFrame:
    import time 
    dataset_copy = df.copy()
    start_time= time.time()
    df['Message'] = df['Message'].apply(preprocess_data)
    df["Message"] = df["Message"].apply(lambda x: remove_emojis(str(x)) if isinstance(x, str) else ' ')
    df = drop_space_rows(df)
    print(df.head(10))
    df.to_csv(concatenated_path, index=False)
    end_time = time.time()

    total_time = end_time - start_time
    print(f"Total time for processing: {total_time:.2f} seconds")

    
    return df

In [ ]:
dataset_copy = dataset.copy() # For visual purposes
dataset = preprocess_dataset(dataset)

b_length = len(dataset_copy)
a_length = len(dataset)
b_mean_length = np.mean(dataset_copy['Message'].str.len())
a_mean_length = np.mean(dataset['Message'].str.len())
b_max_length = np.max(dataset_copy['Message'].str.len())
a_max_length = np.max(dataset['Message'].str.len())
longest_sentence_index = dataset['Message'].str.len().idxmax()
longest_sentence = dataset['Message'].iloc[longest_sentence_index]


print(f"Changes (Before/After) processing:")
print(f"Length: {b_length} -> {a_length}")
print(f"Median length: {b_mean_length:.2f} -> {a_mean_length:.2f}")
print(f"Max sentence length: {b_max_length} -> {a_max_length}")
print(f"Nan values: {dataset_copy.isna().sum().sum()} -> {dataset.isna().sum().sum()}")
print(f"Longest sentence: {len(longest_sentence)} chars: {longest_sentence}")

del dataset_copy

### Diving into Question / Answer 

In [ ]:
"""
Creating a column with time difference between messages 
To correctly assign the context.
"""
dataset = dataset.sort_values(by=['Date']).reset_index(drop=True)

dataset['Date'] = pd.to_datetime(dataset['Date'], format='ISO8601')

reference_time = dataset['Date'].min()
dataset['time_diff_seconds'] = dataset['Date'] - reference_time
# Converts into hours difference
dataset['time_diff_seconds'] = dataset['time_diff_seconds'].apply(lambda x: int(x.total_seconds()))
dataset

In [ ]:
def separate_sentences(df: pd.DataFrame) -> pd.DataFrame:
      """
      Takes a pandas dataframe with a messages column and returns separated rows with question / answer columns
      Args: 
            dataset: pd.DataFrame
            Dataset should contain a messages column and first row with identification who sent a message.



      Returns:
            dataset: pd.DataFrame
            
            Dataset divided into question / answer columns.
      """

      separated_dataset = pd.DataFrame(columns=['question', 'answer', 'timestamp', 'Sent_by_me', 'time_diff_seconds'])

      # Make the first row the first question (All questions become even, answers->odds)
      if df["Sent_by_me"].iloc[0]: 
            df = df.drop(df.index[0]).reset_index(drop=True)

      questions_df = df[df.index % 2 == 0].reset_index(drop=True)
      answers_df = df[df.index % 2 == 1].reset_index(drop=True)

      min_length = min(len(questions_df), len(answers_df))

      separated_dataset = pd.concat(
     [
        questions_df["Message"][:min_length].rename("question"),
        answers_df["Message"][:min_length].rename("answer"),
        df["Date"][:min_length].rename("timestamp"),
        df["Sent_by_me"][:min_length].rename("Sent_by_me"),
        df["time_diff_seconds"][:min_length].rename("time_diff_seconds")
     ], axis=1
)

      return separated_dataset

In [ ]:
dataset = separate_sentences(dataset)


In [ ]:
dataset.head(100)

### Adding column with previous context 

In [ ]:
import pandas as pd
import numpy as np

def add_context(df: pd.DataFrame, context_size: int = 20) -> pd.DataFrame:
    """
    Add a column with previous context to the DataFrame.
    
    The context is based on the previous messages. If the time difference 
    between messages is more than 2 hours, it's considered the start of a 
    new conversation, and the first row of that new conversation will have 
    no context. Subsequent messages in the conversation will have context.
    """
    
    context_list = []
    last_time = None  # Track the last message time to determine time gaps
    
    for index in range(len(df)):
        if index == 0:
            # No context for the very first message
            context_list.append(None)
            last_time = df.loc[index, "time_diff_seconds"]
            continue
        
        # Calculate the time difference from the previous row
        time_diff = df.loc[index, "time_diff_seconds"] - last_time
        last_time = df.loc[index, "time_diff_seconds"]

        # If time_diff is more than 6 hours, consider it a new conversation
        if time_diff > 21600:
            context_list.append(None)  # Start of a new conversation, no context
        else:
            # Create context from the previous messages within the context size
            start_index = max(index - context_size, 0)
            context = df.loc[start_index:index - 1, ["question", "answer"]]

            # Build the context string from previous rows
            message = []
            for key, (question, answer) in enumerate(zip(context["question"], context["answer"])):
                message.append(f"Q{key + 1}: {question}. A{key + 1}: {answer} || ")

            # Append the concatenated message as the context
            context_list.append(" ".join(message))

    # Handle 1st row None (diff seconds in 0 index is 0, then 1 is None).
    context = df.loc[0, ["question", "answer"]]
    question, answer = context["question"], context["answer"]
    context_list[1] = (f"Q{1}: {question}. A{1}: {answer} || ")
    
    # Add the context as a new column
    df["context"] = context_list

    # Replace any empty or missing contexts with "Missing Context" if desired
    df["context"] = df["context"].apply(lambda x: "Time Gap" if pd.isna(x) else x)
    
    return df

In [ ]:
dataset = add_context(dataset)
dataset.head(100)

In [ ]:
total_time_gaps = (dataset["context"] == "Time Gap").sum()
total_time_gaps

# Data Augmentation 
and continue of processing

Resources: 
https://arxiv.org/pdf/1901.11196

Methods: 
1. Back-translation
2. Synonym replacement
3. Word Swap
4. Sentence shuffle

Remember that this notebook is designed to work with ukrainian language dataset, and not all techniques will work for English language.

### Sentence shuffling

In [ ]:
import random
import pandas as pd

def remove_double_commas(text: str) -> str:
    """Removes double commas from the text."""
    return text.replace(",,", ",")

def split_sentences(text: str) -> list:
    """Splits the text into sentences by commas, handling empty strings gracefully."""
    return [sentence.strip() for sentence in text.split(',') if sentence.strip()]

def shuffle_sentence(text: str) -> str:
    """
    Removes double commas, splits the text into sentences, shuffles them,
    and joins them back into a shuffled sentence.
    """
    # Step 1: Clean and split the sentences
    clean_text = remove_double_commas(text)
    sentences = split_sentences(clean_text)

    # Step 2: Shuffle the sentences
    random.shuffle(sentences)

    # Step 3: Join shuffled sentences back into a single string
    return ", ".join(sentences)

Click <b>here</b> for shuffle example.

<!--
text = "This is a test,, sentence, another part,, and more text. Це просто тест, такий вот тест"
shuffled_text = shuffle_sentence(text)

print(f"Before: {text}")
print(f"After: {shuffled_text}")

Outputs: 
Before: This is a test,, sentence, another part,, and more text. Це просто тест, такий вот тест
After: and more text. Це просто тест, another part, This is a test, sentence, такий вот тест
-->

### Back-translation using MarianMTModel
**Not unilizing in the project because of the slow generation time**

Click <b>here</b> to see MarianMTModel


<!--
# Helper function to download data for a language
from transformers import MarianMTModel, MarianTokenizer

def download(model_name):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

# download model for English -> Ukrainian
first_tokenizer, first_model = download('Helsinki-NLP/opus-mt-uk-en')
# download model for Ukrainian -> English
second_tokenizer, second_model = download('Helsinki-NLP/opus-mt-en-uk')

def format_batch_texts(language_code, batch_text):
    formated_batch = [f">>{language_code}<< {batch_text}"]

    return formated_batch

def translate(batch_texts, model, tokenizer, language):
    """Translate texts into a target language"""
    # Format the text as expected by the model
    batched_text = format_batch_texts(language, batch_texts)

    # Translate
    translated = [model.generate(**tokenizer(batch_texts, return_tensors="pt", padding=True)) for sentence in batched_text]

    # Decode (tokens to text)
    translated_texts = tokenizer.batch_decode(translated[0], skip_special_tokens=True)

    return translated_texts

def back_translate(texts, from_language="uk", to_language = "en"):
    """Implements back translation"""
    # Translate from source to target language
    if from_language == "uk": 
        translated = translate(texts, first_model, first_tokenizer, from_language)
        back_translated = translate(translated, second_model, second_tokenizer, to_language)[0]
        return back_translated
    
    translated = translate(texts, second_model, second_tokenizer, from_language)
    back_translated = translate(translated, first_model, first_tokenizer, to_language)[0]

    return back_translated
--->

Click <b>here</b> for back-translation example with MarianMTModel

<!--
# Perform back-translation (Ukrainian to English to Ukrainian)
texts = ["Це перше речення яке ти маєш перекласти.",
         "Воно є дуже просте та правильно сформульованею."]
back_translated_texts = back_translate(texts)
texts = ["This is the first sentence you should translate", 
        "It is simple and correctly formulated"]
back_translated_texts_en = back_translate(texts, "en", "uk")

# Print the results
print("Original Text:", texts)
print("Back-Translated Text:", back_translated_texts)
print("-----------------")
print("Original Text:", texts)
print("Back-Translated Text:", back_translated_texts_en)

Outputs:
Original Text: ['This is the first sentence you should translate', 'It is simple and correctly formulated']
Back-Translated Text: ['Це перше речення, яке ви маєте перекласти.', 'Вона дуже проста і добре сформульована.']
-----------------
Original Text: ['This is the first sentence you should translate', 'It is simple and correctly formulated']
Back-Translated Text: ['This is the first sentence you have to translate.', "It's simple and correctly formulated."]
-->

### Swap and Pop words

In [ ]:
def swap_word(sentence): 
    """Swaps two random words in the sentence"""
    words = word_tokenize(sentence)
    if len(words) < 2:
        return sentence

    idx1, idx2 = np.random.choice(len(words), size=2, replace=False)
    words[idx1], words[idx2] = words[idx2], words[idx1]

    return " ".join(words)

In [ ]:
def filter_stopwords(sentence, stop_words=ukrainian_stop_words) -> str:
    """ Returns two lists: words with stopwords and words without stopwords"""
    words = word_tokenize(sentence)
    filtered_stopwords = [word for word in words if word.lower() not in stop_words]
    return words, filtered_stopwords

def pop_word(sentence, word_swap: bool = False):
    """Pops a random word from the sentence"""

    words, stop_words = filter_stopwords(sentence)

    if stop_words: 
        remove_index = np.random.choice(stop_words, size=1, replace=False)[0]
        words.remove(remove_index)
    else: 
        return sentence


    return " ".join(words)


Click <b>here</b> for word elimination example.

<!--
# Example: 

example = "Це є експериментальним реченням. Воно прикольне))" # Stopwords work only with ukrainian language.
example = pop_word(example)

print(example)

Outputs: 
"is a random sentence"
-->

Click <b>here</b> for word swap example.

<!--
# Example: 

example = "This is a random sentence"
example = swap_word(example)

print(example)
-->

### Back translation and Synonym Replacement

In [ ]:
LANG_CODES = {
    'afrikaans': 'af',
    'albanian': 'sq',
    'amharic': 'am',
    'arabic': 'ar',
    'armenian': 'hy',
    'azerbaijani': 'az',
    'basque': 'eu',
    'belarusian': 'be',
    'bengali': 'bn',
    'bosnian': 'bs',
    'bulgarian': 'bg',
    'catalan': 'ca',
    'cebuano': 'ceb',
    'chichewa': 'ny',
    'chinese (simplified)': 'zh-cn',
    'chinese (traditional)': 'zh-tw',
    'corsican': 'co',
    'croatian': 'hr',
    'czech': 'cs',
    'danish': 'da',
    'dutch': 'nl',
    'english': 'en',
    'esperanto': 'eo',
    'estonian': 'et',
    'filipino': 'tl',
    'finnish': 'fi',
    'french': 'fr',
    'frisian': 'fy',
    'galician': 'gl',
    'georgian': 'ka',
    'german': 'de',
    'greek': 'el',
    'gujarati': 'gu',
    'haitian creole': 'ht',
    'hausa': 'ha',
    'hawaiian': 'haw',
    'hebrew': 'he',
    'hindi': 'hi',
    'hmong': 'hmn',
    'hungarian': 'hu',
    'icelandic': 'is',
    'igbo': 'ig',
    'indonesian': 'id',
    'irish': 'ga',
    'italian': 'it',
    'japanese': 'ja',
    'javanese': 'jw',
    'kannada': 'kn',
    'kazakh': 'kk',
    'khmer': 'km',
    'korean': 'ko',
    'kurdish (kurmanji)': 'ku',
    'kyrgyz': 'ky',
    'lao': 'lo',
    'latin': 'la',
    'latvian': 'lv',
    'lithuanian': 'lt',
    'luxembourgish': 'lb',
    'macedonian': 'mk',
    'malagasy': 'mg',
    'malay': 'ms',
    'malayalam': 'ml',
    'maltese': 'mt',
    'maori': 'mi',
    'marathi': 'mr',
    'mongolian': 'mn',
    'myanmar (burmese)': 'my',
    'nepali': 'ne',
    'norwegian': 'no',
    'odia': 'or',
    'pashto': 'ps',
    'persian': 'fa',
    'polish': 'pl',
    'portuguese': 'pt',
    'punjabi': 'pa',
    'romanian': 'ro',
    'russian': 'ru',
    'samoan': 'sm',
    'scots gaelic': 'gd',
    'serbian': 'sr',
    'sesotho': 'st',
    'shona': 'sn',
    'sindhi': 'sd',
    'sinhala': 'si',
    'slovak': 'sk',
    'slovenian': 'sl',
    'somali': 'so',
    'spanish': 'es',
    'sundanese': 'su',
    'swahili': 'sw',
    'swedish': 'sv',
    'tajik': 'tg',
    'tamil': 'ta',
    'telugu': 'te',
    'thai': 'th',
    'turkish': 'tr',
    'ukrainian': 'uk',
    'urdu': 'ur',
    'uyghur': 'ug',
    'uzbek': 'uz',
    'vietnamese': 'vi',
    'welsh': 'cy',
    'xhosa': 'xh',
    'yiddish': 'yi',
    'yoruba': 'yo',
    'zulu': 'zu'}

LANGUAGES = {value:key for key, value in LANG_CODES.items()}

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
import psutil

from nlpaug.util import Action

aug_glove = naw.WordEmbsAug(
    model_type='glove', model_path='Models/glove.6B.100d.txt',
    action="substitute")

In [ ]:
from googletrans import Translator
import gensim.downloader as api


class google_translate:
    """
    Performs Google Translate on a given text.

    Args:
        translate_from (str): The natural language of the text. Defaults to "uk". Contains auto language detection.
        translate_to (str): The language to translate to and back from. Defaults to "en".
    """

    def __init__(self, translate_from: str = "uk", translate_to: str = "en", replace_synonyms: bool = False):
        self.native_language = translate_from
        self.tunnel_language = translate_to 
        self.translator = Translator()

        if replace_synonyms:
            self.word2vec_model = self.install_word2vec()

    """ Back-translation """
    # Check whether the language input is correct
    @cache
    def check_language(self, text):
        try: 
            if self.native_language not in LANGUAGES:  
                self.native_language = self.translator.detect(text).lang
                print(f"Incorrect language. Translating from '{self.native_language}'")

                # If the back-translation is going on English text, the text will be translated from English to Spanish and back to English.
                if self.native_language == "en": 
                    self.tunnel_language = "es"

        except Exception as e:
            raise Exception("Check_language: " + str(e))

    @cache 
    def back_translate(self, text, replace_synonym: bool = True) -> str:
        """
        Performs back-translation on a given text.

        Args:
            text (str): The text to back-translate.
            temp_lang (str): The intermediate language for translation. Defaults to French ("fr").

        Returns:
            str: The back-translated text.
        """
        translator = self.translator
        try: 
            self.check_language(text=text)

            translated = self.translator.translate(text, src=self.native_language, dest=self.tunnel_language).text
            
            if replace_synonym: 
                translated = self.synonym_replacement(sentence=translated) 

            back_translated = translator.translate(translated, src=self.tunnel_language, dest=self.native_language).text

            return back_translated
        except Exception as e: 
            print("back_translate: Something went wrong.")

    """ Synonym extension (Word2Vec) """

    

    def install_word2vec(self):
      model_name = "word2vec-google-news-300"
      print(f"Configuring {model_name}")
      word2vec_model = api.load(model_name)

      return word2vec_model

    @cache
    def synonym_replacement(self, sentence, percentage: float = 0.7): 
        """ Replaces random non-stopword word with a synonym. 

        Args:
            percentage (float, optional): Percentage of words to replace. Defaults to 0.7.
        
        """
        # Remove stopwords 
        words, filtered_sentence = filter_stopwords(sentence)
        if words: 
            try: 
                random_word_index = np.random.choice(len(filtered_sentence), size=int(percentage * len(filtered_sentence) if len(filtered_sentence) > 1 else 1))[0]
                word_to_replace = filtered_sentence[random_word_index]
                synonym = self.word2vec_model.most_similar(word_to_replace, topn=1)[0][0] # Top 5 most similar words
                # Fill the chosen word for a synomym
                for idx, word in enumerate(words): 
                    if word == word_to_replace: 
                        words[idx] = word_to_replace

                return " ".join(words)
            except Exception as e: 
                print("synonym_replacement Exception: Could not replace synonym.")
                return sentence        


Click <b>here</b> to see back-translation example.


<!--
translator = google_translate(translate_from="uk", translate_to="en")
back_translated = translator.back_translate("Привіт, як воно?") # Hello, how is it going?
back_translated
--->

### Creating augmentation rows and concatenating them with dataset.

Click <b>here</b> for short augmentation example 

<!--
translator = google_translate(translate_from="uk", translate_to="en", replace_synonyms=True)

# Short implementation of random augmentation
augmentation_functions = [translator.back_translate, shuffle_sentence, pop_word, swap_word]
indexes = np.random.choice(len(augmentation_functions), size=random.randint(1, 4), replace=False)   
functions = [augmentation_functions[index] for index in sorted(indexes)]

sentence = "Example"
print(functions)
for function in functions: 
    sentence = function(sentence)
sentence
---->

In [ ]:
translator = google_translate(translate_from="uk", translate_to="en", replace_synonyms=True)

In [ ]:
def is_memory(threshold_gb: float = 4, delay: int = 10): 
    """
    Pauses execution when available memory is less than threshold.
    Args:f
    - threshold_gb (float): Max memory allowed in GB.
    - delay (int): Seconds to wait before rechecking memory.
    """
    available_ram = psutil.virtual_memory().available / (1024**3)
    if available_ram <= threshold_gb:
        print(available_ram)
        #print("Memory limit reached. Waiting for resources to free up...")
        time.sleep(delay)

In [ ]:
augmentation_functions = [translator.back_translate, shuffle_sentence, pop_word, swap_word]
def select_random_functions(functions=augmentation_functions, p=[0.1, 0.3, 0.3, 0.3]):  # Lowered probabilities for back-translation because of low-resources
    """ Returns random functions in order to apply during processing"""

    indexes = sorted(np.random.choice(len(functions), size=random.randint(1, len(functions)), replace=False, p=p))
    return [functions[index] for index in indexes]            

def apply_augmentation(sentence, random_augmentation: bool = True) -> pd.DataFrame:
    try: 
        # Check for available memory 
        is_memory()

        if random_augmentation:
            functions = select_random_functions()
            for function in functions:
                sentence = function(sentence)
            return sentence 
        
        sentence = translator.back_translate(sentence, replace_synonym=True)
        sentence = shuffle_sentence(sentence)
        sentence = swap_word(sentence)
        sentence = pop_word(sentence)
    except Exception as e: 
        print("apply_augmentation EXCEPTION: " + str(e))
        return sentence

def speed_test(df, samples: int = 100) -> None:
    start_time = time.time()
    df["question"] = df["question"][:samples].apply(lambda x: apply_augmentation(x))
    print("--- %s seconds ---" % (time.time() - start_time))
    return 

def augment_data(df: pd.DataFrame, 
                save_path: str = None,
                augmentation_factor: int = 5, 
                random_augmentation: bool = False, 
                samples: int = None, 
                checkpoints: bool = True) -> pd.DataFrame:

    """
    Augments the data by adding augmented questions.
    
    Parameters:
        df: pd.DataFrame with "question" column
        augmentation_factor: int = 5; how many times to augment each question.
        samples: int = None; How much rows to process. 
        checkpoints: bool = True; Saves the augmentation process every iteration (augmentation_factor==1Iter)
    """
    original_dataframe = df[:200]
 

    df_augmented = original_dataframe.copy()
    df_augmented = drop_space_rows(df_augmented, column="question")
    for _ in tqdm(range(augmentation_factor)):
        loop_dataset = original_dataframe.copy()
        loop_dataset["question"] = loop_dataset["question"].apply(lambda x: apply_augmentation(x, random_augmentation=random_augmentation))
    
        df_augmented = pd.concat([df_augmented, loop_dataset], axis=0).reset_index(drop=True)

        if checkpoints: 
            # For notebook
            #folder_path = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..', 'Datasets')) 
            #save_path = os.path.join(folder_path, 'instagram_data.csv')
            if not save_path:
                save_path = "Datasets/dataset"
                df_augmented.to_csv(save_path, index=False)
                continue
            
            # For the parallel augmentation
            # Path would be chunks/chunk_1 , chunks/chunk_2 etc..
            df_augmented.to_csv(save_path, index=False)
    
    # Sort the dataset for sequential data.
    df_augmented = df_augmented.sort_values(by='timestamp').reset_index(drop=True)
    df_augmented.drop_duplicates(inplace=True)

    return df_augmented


In [ ]:
#augmented_100 = augment_data(dataset, random_augmentation=True, samples=200) # Optimization is not used

### Setting up efficient processing with various optimization techniques

Before starting processing our datasets, we will work on the optimization and speeding of our code. Since the processing functions requires a lot of computations, we will work on it.

* Added cache to avoid performing computations multiple times
* Added parallel processing
* Added auto memory managment

In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait
import multiprocess as mp # NOT multiprocessing to avoid __main__ improtable problem by the children 
from functools import partial

In [ ]:
def split_dataframe(df, chunk_size = os.cpu_count()):
    chunks = np.array_split(df, chunk_size)
    return chunks

def process_parallel(df_chunk, **kwargs):
    return augment_data(df_chunk, **kwargs)

df_chunks = split_dataframe(dataset)
kwargs: dict = {
    "augmentation_factor": 2,
    "random_augmentation": True,
    "samples": None, 
    "checkpoints": True}


In [ ]:
def augmentation_wrapper(df: pd.DataFrame, save_path: str, **kwargs):
      return augment_data(df, save_path, **kwargs)

def parallel_computing(df, func, num_partitions=mp.cpu_count()-2, **kwargs):
    df_split = np.array_split(df, num_partitions) 
    save_paths = [f"data_chunks/chunk_{i+1}" for i in range(num_partitions)] # Create save_paths for each partition
    
    func_with_kwargs = partial(func, **kwargs)

    # Create a pool of workers
    pool = mp.Pool(num_partitions)
    try:
      # Apply the function to each partition in parallel
      pool.starmap(partial(func, **kwargs), [(df_split[i], save_paths[i]) for i in range(num_partitions)])
      pool.join()
      
      return df
    # Close the pool and wait for the work to finish
    except Exception as e:
      print("parallel_computing EXCEPTION: " + str(e))
      pool.terminate()

      pool.join()
  

In [ ]:
augmented_dataset = parallel_computing(dataset, augmentation_wrapper, **kwargs)

In [ ]:
# Stop all processings
pool = mp.Pool(mp.cpu_count()-2)
pool.terminate()
pool.join()
print("All workers have been terminated.")

In [ ]:
# Let's compare the speed of regular vs parallel processing
test_samples = 400


# Speed test
regular_start = time.time()
augmented_dataset = augment_data(dataset, random_augmentation=True, samples=test_samples)
regular_finish = time.time()

parallel_start = time.time()
kwargs["samples"] = test_samples
#kwargs["samples"] = test_samples / (mp.cpu_count()-2)
augmented_dataset = parallel_computing(dataset, augmentation_wrapper, **kwargs)
parallel_finish = time.time()

print(f"Regular: {regular_finish - regular_start:.2f} seconds")
print(f"Parallel: {parallel_finish - parallel_start:.2f} seconds")

### Batches